In [ ]:
import os
import sys
import glob
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

#### Load the network and the best weights

In [ ]:
networks_path = '../experiments/'

# depth level = 1
# loss function: MAPE
# omega: COI
# optim_run = '20201206-221201'

# depth level = 1
# loss function: MAE
# omega: COI
# optim_run = '20201206-221238'

# depth level = 1
# loss function: MAE
# omega: G1
optim_run = 'd7f30fef5806435897e97f27606fa30a'

checkpoint_path = networks_path + optim_run + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
network_parameters = pickle.load(open(networks_path + optim_run + '/parameters.pkl', 'rb'))
val_loss = [float(file[:-3].split('-')[-1]) for file in checkpoint_files]
best_checkpoint = checkpoint_files[np.argmin(val_loss)]
best_model = tf.keras.models.load_model(best_checkpoint, compile=True)
if os.path.isdir('../' + network_parameters['data_dir']):
    data_dir = '../' + network_parameters['data_dir']
else:
    data_dir = '../data/IEEE14_D=0_DZA=0.36'
print('Loaded network from {}.'.format(best_checkpoint))
print('Data directory is {}.'.format(data_dir))

#### We need mean and standard deviation of the training set to normalize the data

In [ ]:
try:
    x_train_mean = network_parameters['x_train_mean']
    x_train_std  = network_parameters['x_train_std']
    print('Loaded mean and std of training set from stored parameters.')
except:
    # Load the data used to train the network to compute the mean
    # and standard deviation of the training set if this is not
    # contained in the parameters file
    print('Extracting mean and std of training set from data in {}.'.format(data_dir))
    inertia = {'training': np.arange(2,11)}
    _, x, y = load_data(data_dir, inertia)
    x['train'] = x.pop('training')
    y['train'] = y.pop('training')
    x_train_mean = np.mean(x['train'])
    x_train_std = np.std(x['train'])

In [ ]:
best_model.summary()

#### A couple of functions used in the following

In [ ]:
def load_data(data_files, var_name='omegacoi', window_dur = 60, window_step = 10, verbose=False):
    
    data = [np.load(data_file, allow_pickle=True) for data_file in data_files]
    time = [data[0]['time']]
    for i in range(1,len(data)):
        time.append(data[i]['time'] + data[i-1]['time'][-1])
    time = np.concatenate(time)
    N_samples = time.size
    omega = np.concatenate([entry[var_name] for entry in data])
    omega_normalized = (omega - x_train_mean) / x_train_std

    try:
        simulation_parameters = [d['parameters'].item() for d in data]
    except:
        simulation_parameters = [{'frand': 10} for _ in range(len(data_files))]

    dt = 1 / simulation_parameters[0]['frand']
    window_size = int(window_dur / dt)
    if verbose:
        print('Window size: {:d} samples'.format(window_size))

    omega_sliding, indexes = slide_window(omega_normalized, window_size, window_step=window_step)
    if verbose:
        print('Number of trials: {:d}'.format(omega_sliding.shape[0]))
    
    return time, omega, omega_sliding, indexes, simulation_parameters


def predict(omega_sliding, model, window_step, dt, rolling_length=50):
    x = tf.constant(omega_sliding, dtype=tf.float32)
    y = np.squeeze(model.predict(x))
    H = pd.DataFrame(data = {'inertia': y}).rolling(rolling_length).mean().to_numpy()
    time = np.arange(H.size) * window_step * dt
    return time, H, y

### First example
1. H steps **instantaneously** from 3.5 to 8.5
1. **omega of the center of inertia** used for the estimation
1. two simulations joined together (i.e., **no transient** during the step)

In [ ]:
H_values = [3.5, 8.5]
data_file_fmt = data_dir + '/ieee14_H={}.npz'
data_files = [data_file_fmt.format(h) for h in H_values]
data = [np.load(data_file, allow_pickle=True) for data_file in data_files]

In [ ]:
var_names = ('omegacoi', 'omega01', 'omega02', \
             'G3:omega', 'G6:omega', 'G8:omega')
n_vars = len(var_names)
col = 'kr'
fig,ax = plt.subplots(n_vars, 2, figsize=(10, 2.5*n_vars))
align = ['center', 'edge']
n_bins = 25
for i,var_name in enumerate(var_names):
    for j,d in enumerate(data):
        t = d['time']
        x = d[var_name]
        idx = t < 120
        x_norm = (x - x_train_mean) / x_train_std
        n,edges = np.histogram(x_norm, bins=n_bins, range=[-4,4], density=True)
        ax[i,0].plot(t[idx], x[idx], color=col[j], linewidth=1)
        ax[i,1].bar(edges[:-1], n, width=np.diff(edges)[0] / 2, align=align[j],
                    color=col[j], linewidth=2, edgecolor=col[j], alpha=0.5)
for i in range(ax.shape[0]):
    ax[i,0].set_ylabel(r'$\omega_{{{}}}$'.format(var_names[i].replace('omega','').replace(':','').replace('0','G')))
    ax[i,1].set_ylabel('Fraction')
ax[-1,0].set_xlabel('Time [s]')
ax[-1,1].set_xlabel(r'Normalized $\omega$')
fig.tight_layout()

In [ ]:
var_names = ('omegacoi', 'omega01', 'omega02', \
             'G3:omega', 'G6:omega', 'G8:omega')
real_inertia_values = {
    'omegacoi': H_values,
    'omega01': H_values,
    'omega02': 13.08 / 2 + np.zeros(2),
    'G3:omega': 13.08 / 2 + np.zeros(2),
    'G6:omega': 10.12 / 2 + np.zeros(2),
    'G8:omega': 10.12 / 2 + np.zeros(2)
}
window_dur = 60
window_step = 30

fig,ax = plt.subplots(3, 2, figsize=(10,10), sharex=True, sharey=True)
for n,var_name in enumerate(var_names):
    t, _, omega_sliding, indexes, sim_pars = load_data(data_files, var_name, window_dur, window_step)
    dt = np.diff(t[:2])[0]
    time, H, _ = predict(omega_sliding, best_model, window_step, dt)
    i = n // ax.shape[1]
    j = n % ax.shape[1]
    ax[i,j].plot(time[[0,int(time.size/2)]], \
                 real_inertia_values[var_name][0] + np.zeros(2), \
                 'r--', lw=2, label='Exact')
    ax[i,j].plot(time[[int(time.size/2),-1]], \
                 real_inertia_values[var_name][1] + np.zeros(2), \
                 'r--', lw=2)
    ax[i,j].plot(time, H, 'k', lw=1, label='Estimated')
    ax[i,j].set_title(var_name.replace('omega','').replace(':','').replace('0','G').upper())

for i in range(ax.shape[1]):
    ax[-1,i].set_xlabel('Time [s]')
for i in range(ax.shape[0]):
    ax[i,0].set_ylabel(r'Inertia')
ax[0,0].set_ylim([0,11])
ax[0,0].legend(loc='lower right')
fig.tight_layout();

### Second example
1. H steps **instantaneously** from 3.5 to 8.5
1. **omega of generator 1** used for the estimation
1. two simulations joined together (i.e., **no transient** during the step)

### Third example
1. H steps **gradually** from 3.5 to 6.5 in 100 seconds (from t = 3550s to t=3650s)
1. **omega of generator 1** used for the estimation

#### TODO
1. D=2, DZA=1: damping, super-wide dead-band
1. D=2, DZA=36e-3/60: damping, realistic dead-band
1. D=0, DZA=0.36/60: no damping, wide dead-band